In [ ]:
! pip install umap-learn

In [ ]:
import pandas as pd
from umap import UMAP

In [ ]:
SQL = "SELECT * from `questrom.datasets.airline-intents` where intent in ('atis_airfare', 'atis_ground_service', 'atis_airline', 'atis_abbreviation')"
msgs = pd.read_gbq(SQL, "questrom")

In [ ]:
msgs.head(3)

In [ ]:
# vectorize the data
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



In [ ]:
# vectorize the data
tf = TfidfVectorizer()
tf.fit(msgs.text)

dtm = tf.transform(msgs.text)




In [ ]:
type(dtm)

In [ ]:
dtm.shape

In [ ]:
# umap for DR

umap = UMAP(random_state=820)
embeds = umap.fit_transform(dtm)


In [ ]:
type(embeds)

In [ ]:
embeds.shape

In [ ]:
msgs.shape

In [ ]:
# scatterplot for viz
import seaborn as sns

In [ ]:
# plot it up
sns.scatterplot(x=embeds[:, 0], y=embeds[:, 1], hue=msgs.intent)


In [ ]:
##  there is a table on big query
##  questrom.datasets.bruins_twitter
##
##  get the records where the hour is 0,1,2,3
##  this is not a select *, you have to filter records
##  - TRICKY: apply afinn sentiment to each record
##  - ensure that the data sorted by status_id
##  - plot the sentiment score over the records (this is a timeseries - like view)
##  - calculate the average sentiment by hour
##
##

SQL = "SELECT * from `questrom.datasets.bruins_twitter` where hour in (0,1,2,3)"
tweets = pd.read_gbq(SQL, "questrom")

In [ ]:
# a quick look
tweets.sample(3)

In [ ]:
# info of the dataset
tweets.info()

In [ ]:
! pip install afinn

In [ ]:
# apply the sentiment over all of the records
from afinn import Afinn

afinn = Afinn()

# function
def sentiment(text):
  return afinn.score(text)



In [ ]:
# apply the sentiment score
tweets['sent'] = tweets.text.apply(sentiment)

In [ ]:
tweets.head(3)

In [ ]:
# describe
tweets.sent.describe()

In [ ]:
# plot sentiment over time
tweets.sort_values("status_id", ascending=True, inplace=True)

# properly handle the datetime
tweets['created'] = pd.to_datetime(tweets.created_at)

In [ ]:
# plot
sns.lineplot(x=tweets.index, y=tweets.sent)


In [ ]:
sns.lineplot(x=tweets.created, y=tweets.sent)

In [ ]:
# aggregate by hour
tweets.groupby("hour").agg({'sent':['size', 'mean']})